In [142]:
import numpy as np

import matplotlib.pyplot as plt

from random import random
from copy import deepcopy

In [143]:
X = np.reshape([random()*100 for i in range(400)],(100,4))

In [144]:
col_max = np.amax(X,axis=0)
col_min = np.amin(X,axis=0)
centroid = (col_max + col_min)/2

In [145]:
Z = (X-centroid)**2

In [146]:
alpha = (1.0 / 100) * np.ones(100)

In [147]:
ZT = Z.T
h = np.matmul(alpha,Z)
h

array([812.00161162, 818.19143272, 762.64232322, 702.53165298])

In [148]:
def isclose(a, b, rel_tol, abs_tol=0.0):
    return abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)

In [149]:
def calc_s(h,alpha_old,Z,u,i):
  ans=0
  for j in range(4):
            s_i_j = h[j] - alpha_old[i]*Z[i][j]
            ans += (1/(u + s_i_j/Z[i][j]))
  return ans

In [150]:
def solution(Z,alpha,h,tol):
    i = 0
    updates = 0
    converged = False
    alpha_old = np.zeros(100)
    old = 0
    start=0

    
    while i < 100:
        if converged:
            return h
        alpha_old[i] = alpha[i]
        f_x = 0
        u = random()
        f_x=calc_s(h,alpha_old,Z,u,i)
        
        if f_x == 1:
            alpha[i] = u
        
        if f_x < 1:

            alpha[i] = 0

        if f_x > 1:
            u+=0.00001*u   # this 0.00001 is step which i am decreasing the u for binary search..
            while(not isclose(f_x,1,tol)):
                f_x = calc_s(h,alpha_old,Z,u,i)
                if(f_x>1):
                    u+=0.0001*u
                else:
                    u-=0.0001*u 
            alpha[i]=u    
            
                
        h = h + (alpha[i] - alpha_old[i])*Z[i]
        i += 1
        if i>=99 :
            i = 0
            updates += 1
            if old == 0:
                stored_alpha_old = deepcopy(alpha_old)
                old = 1
        if updates == 4:
            if np.all(alpha - stored_alpha_old) < tol :  #.00001
                converged = True
            else:
                updates = 0
                stored_alpha_old = deepcopy(alpha)
    

In [151]:
h = solution(Z,alpha,h,1e-05)
h

array([7793.66404336, 8790.59236428, 5925.49807875, 4505.72566006])

## weight vectors obtained are:

In [152]:
v = 1/h
v

array([0.00012831, 0.00011376, 0.00016876, 0.00022194])

## visualising the ellipsoid with data points


In [153]:
def randrange(n, vmin, vmax):
    """
    Helper function to make an array of random numbers having shape (n, )
    with each number distributed Uniform(vmin, vmax).
    """
    return (vmax - vmin)*np.random.rand(n) + vmin

In [154]:
ax=plt.axes(projection='3d')
# ax.set_xlim(0,150)
# ax.set_ylim(0,150)
# ax.set_zlim(0,150)
ax.scatter3D(X[:,0],X[:,1],X[:,2],s=30)
ax.set_xlabel('first feature')
ax.set_ylabel('second feature')
ax.set_zlabel('third feature')
ys = randrange(40, 0, 100)
zs = randrange(40, 0,100)
xs = randrange(40, 0, 100)
ax.scatter(xs, ys, zs, marker='^');